# ಆಟೋಜೆನ್ ಸಾಧನ ಬಳಕೆ ಉದಾಹರಣೆ


## ಅಗತ್ಯವಿರುವ ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದುಮಾಡಿ


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## ಕ್ಲೈಂಟ್ ರಚನೆ 

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ಅನ್ನು LLM ಗೆ ಪ್ರವೇಶಿಸಲು ಬಳಸುತ್ತೇವೆ. 

`model` ಅನ್ನು `gpt-4o-mini` ಎಂದು ವ್ಯಾಖ್ಯಾನಿಸಲಾಗಿದೆ. GitHub Models ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಲಭ್ಯವಿರುವ ಇನ್ನೊಂದು ಮಾದರಿಗೆ ಮಾದರಿಯನ್ನು ಬದಲಾಯಿಸಲು ಪ್ರಯತ್ನಿಸಿ ಮತ್ತು ವಿಭಿನ್ನ ಫಲಿತಾಂಶಗಳನ್ನು ನೋಡಿ. 

ಒಂದು ತ್ವರಿತ ಪರೀಕ್ಷೆಯಾಗಿ, ನಾವು ಸರಳ ಪ್ರಾಂಪ್ಟ್ ಅನ್ನು ನಡೆಸುತ್ತೇವೆ - `What is the capital of France`. 


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ಕಾರ್ಯಗಳನ್ನು ನಿರ್ಧರಿಸುವುದು

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು ಏಜೆಂಟ್‌ಗೆ ಒಂದು ಸಾಧನವನ್ನು ಪ್ರವೇಶಿಸಲು ಅವಕಾಶ ನೀಡುತ್ತೇವೆ, ಅದು ಲಭ್ಯವಿರುವ ರಜಾ ಸ್ಥಳಗಳ ಪಟ್ಟಿ ಮತ್ತು ಅವುಗಳ ಲಭ್ಯತೆಯನ್ನು ಹೊಂದಿರುವ ಕಾರ್ಯವಾಗಿದೆ.

ನೀವು ಇದನ್ನು ಒಂದು ಪ್ರಯಾಣ ಏಜೆಂಟ್‌ಗೆ ಪ್ರಯಾಣ ಡೇಟಾಬೇಸ್‌ಗೆ ಪ್ರವೇಶವಿರುವ ಪರಿಸ್ಥಿತಿಯಂತೆ ಕಲ್ಪಿಸಬಹುದು.

ಈ ಮಾದರಿಯನ್ನು ನೋಡುತ್ತಾ ಹೋಗುವಾಗ, ಏಜೆಂಟ್ ಕರೆ ಮಾಡಬಹುದಾದ ಹೊಸ ಕಾರ್ಯಗಳು ಮತ್ತು ಸಾಧನಗಳನ್ನು ನಿರ್ಧರಿಸಲು ಪ್ರಯತ್ನಿಸಬಹುದು.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## ಫಂಕ್ಷನ್ ಟೂಲ್ ಅನ್ನು ನಿರ್ಧರಿಸುವುದು  
`vacation_destinations` ಅನ್ನು `FunctionTool` ಆಗಿ ಏಜೆಂಟ್ ಬಳಸಲು, ನಾವು ಅದನ್ನು ಒಂದು ಫಂಕ್ಷನ್ ಟೂಲ್ ಆಗಿ ನಿರ್ಧರಿಸಬೇಕು.  

ನಾವು ಟೂಲ್‌ಗೆ ಒಂದು ವಿವರಣೆಯನ್ನು ಸಹ ಒದಗಿಸುತ್ತೇವೆ, ಇದು ಬಳಕೆದಾರನು ಕೇಳಿದ ಕಾರ್ಯಕ್ಕೆ ಸಂಬಂಧಿಸಿದಂತೆ ಆ ಟೂಲ್ ಏನಿಗಾಗಿ ಬಳಸಲಾಗುತ್ತದೆ ಎಂಬುದನ್ನು ಏಜೆಂಟ್ ಗುರುತಿಸಲು ಸಹಾಯಕವಾಗುತ್ತದೆ.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## ಏಜೆಂಟ್ ಅನ್ನು ನಿರ್ಧರಿಸುವುದು

ಈಗ ನಾವು ಕೆಳಗಿನ ಕೋಡ್‌ನಲ್ಲಿ ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸಬಹುದು. ಬಳಕೆದಾರರಿಗೆ ರಜಾ ಸ್ಥಳಗಳನ್ನು ಹುಡುಕಲು ಸಹಾಯ ಮಾಡುವ ಬಗ್ಗೆ ಏಜೆಂಟ್‌ಗೆ ಸೂಚನೆಗಳನ್ನು ನೀಡಲು ನಾವು `system_message` ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತೇವೆ.

ನಾವು `reflect_on_tool_use` ಪ್ಯಾರಾಮೀಟರ್ ಅನ್ನು true ಗೆ ಹೊಂದಿಸುತ್ತೇವೆ. ಇದು ಟೂಲ್ ಕಾಲ್‌ನ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ತೆಗೆದು, ನೈಸರ್ಗಿಕ ಭಾಷೆಯನ್ನು ಬಳಸಿಕೊಂಡು ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಕಳುಹಿಸಲು LLM ಅನ್ನು ಬಳಸಲು ಅನುಮತಿಸುತ್ತದೆ.

ನೀವು ಈ ಪ್ಯಾರಾಮೀಟರ್ ಅನ್ನು false ಗೆ ಹೊಂದಿಸಿ ವ್ಯತ್ಯಾಸವನ್ನು ನೋಡಬಹುದು.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸುವುದು

ಈಗ ನಾವು ಟೋಕಿಯೋಗೆ ಪ್ರಯಾಣ ಮಾಡಲು ಕೇಳುವ ಪ್ರಾಥಮಿಕ ಬಳಕೆದಾರ ಸಂದೇಶದೊಂದಿಗೆ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸಬಹುದು.

ನೀವು ಈ ನಗರ ಗಮ್ಯಸ್ಥಾನವನ್ನು ಬದಲಾಯಿಸಿ, ಆ ನಗರ ಲಭ್ಯತೆಯ ಬಗ್ಗೆ ಏಜೆಂಟ್ ಹೇಗೆ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಬಹುದು.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಾಖಲೆ AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅನ್ವಯಗಳುಗಾಗಿ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
